### Tecnolgías del Lenguaje. Entregable 4
---
#### Preprocesamiento de datos

Dividir los posts en oraciones permite un análisis lingüístico más preciso y granular, ya que las oraciones son unidades semánticas más estables que los textos completos. En el procesamiento del lenguaje natural, trabajar a nivel de oración facilita tareas como la detección de rasgos de personalidad, el análisis sintáctico o la extracción de características estilísticas, evitando que la longitud o la complejidad de los textos afecten al modelo. Además, al segmentar los posts, se pueden identificar patrones de escritura y estilo más coherentes y comparables entre distintos autores, mejorando la calidad y la interpretabilidad de los resultados en el análisis posterior.

Por ello, en este notebook realizamos la **segmentación de los posts por oraciones**, transformando cada entrada textual en una lista de frases individuales. Este paso prepara el dataset para etapas posteriores de análisis lingüístico y de personalidad, en las que cada oración podrá ser tratada como una unidad de observación independiente. De esta forma, se facilita la extracción de rasgos lingüísticos específicos, la aplicación de modelos de NLP basados en contexto y la identificación de patrones de escritura más finos y consistentes entre los distintos usuarios.

Queremos transformar el dataset original en uno que contenga una fila por usuario y una lista para cada usuario que almacene las oraciones de todos sus posts. El objetivo es preparar un dataset que sirva para entrenar un modelo Transformer con el que reduzcamos el gran conjunto de datos original al conjunto de datos de interés, esto es, aquel que **almacena las frases de cada usuario que son más representativas de cada rasgo Big-5**.

- **Input: material proporcionado en bruto.**
- **Output: material preprocesado, preparado para ser procesado por un Transformer.**
---

In [1]:
# Requisitos
# pip install pandas emoji ftfy clean-text emot
import re
import pandas as pd
import emoji
from ftfy import fix_text
from cleantext import clean
from emot.emo_unicode import EMOTICONS  # para emoticonos tipo :-) :P

# Mapa de símbolos/pictogramas extra
SYMBOL_MAP = {
    '♬': 'music', '♪': 'musical_note', '♫': 'melody',
    '♥': 'heart', '❤': 'heart', '💕': 'hearts', '💔': 'broken_heart',
    '✨': 'shine', '⭐': 'star', '🌟': 'star',
    '✔': 'check', '✖': 'cross', '…': 'ellipsis'
}

# Construir un mapa de emoticonos (texto) a tokens legibles
EMOTICON_MAP = {}
for k, v in EMOTICONS.items():
    # EMOTICONS keys are regex-like; normalizamos claves simples
    safe_key = re.sub(r'[^A-Za-z0-9]', '_', k)[:40]
    EMOTICON_MAP[k] = v.replace(',', '').replace(':','').strip().replace(' ', '_')

# Rango general para símbolos/pictogramas unicode (capturamos los que no están en emoji.demojize)
UNICODE_SYMBOLS_RE = re.compile(
    "[" 
    "\U0001F300-\U0001F5FF"  # símbolos y pictogramas
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transporte y símbolos varios
    "\u2600-\u26FF"          # misc symbols (♬, ❤ etc)
    "]+", flags=re.UNICODE
)

URL_RE = re.compile(r'(https?://\S+|www\.\S+)')
EMAIL_RE = re.compile(r'\b[\w\.-]+@[\w\.-]+\.\w{2,}\b')
MENTION_RE = re.compile(r'@\w+')
HASHTAG_RE = re.compile(r'#(\w+)')
MULTI_PUNCT_RE = re.compile(r'([!?.]){2,}')  # !!! o ???
NON_PRINTABLE_RE = re.compile(r'[\x00-\x1f\x7f-\x9f]')
EXCESS_WS_RE = re.compile(r'\s+')

def replace_symbol_tokens(text):
    for s, w in SYMBOL_MAP.items():
        text = text.replace(s, f' {w} ')
    return text

def replace_emoticons(text):
    # Las claves de EMOTICONS keys no siempre se pillan
    for emoticon, meaning in EMOTICON_MAP.items():
        # intenta reemplazar directamente los emoticones ascii del diccionario de EMOTICONS
        if emoticon in text:
            text = text.replace(emoticon, f' {meaning} ')
    # fallback: captura patrones simples como :-), :P, XD, :(
    text = re.sub(r'(:-\)|:\)|:\]|=+\)|:D|XD|xD)', ' :smiling_face ', text, flags=re.IGNORECASE)
    text = re.sub(r'(:-\(|:\()', ' :sadness_face ', text, flags=re.IGNORECASE)
    text = re.sub(r'(:P|:p|;P|;p)', ' :sticking_out_tongue ', text, flags=re.IGNORECASE)
    return text

def demojize_and_map(text):
    # demojize convierte emojis a :emoji_name:
    text = emoji.demojize(text, language='es')  # devuelve :smiling_face:
    # convertir :emoji_name: -> emoji_name (sin dos puntos, con guiones -> guiones)
    text = re.sub(r':([a-zA-Z0-9_+-]+):', lambda m: ' ' + m.group(1).replace('_', ' ') + ' ', text)
    return text

def remove_junk_chars(text):
    # reemplaza caracteres no imprimibles y secuencias de puntuación repetidas (reduce '!!!' a ' ! ')
    text = NON_PRINTABLE_RE.sub(' ', text)
    text = MULTI_PUNCT_RE.sub(lambda m: ' ' + m.group(1) + ' ', text)
    return text

def clean_post(text, lower=True, keep_hashtags=False):
    if not isinstance(text, str):
        return ''
    # unicode fixes (arregla mojibake y cosas raras)
    text = fix_text(text)
    # quitar URLs y correos
    text = URL_RE.sub(' ', text)
    text = EMAIL_RE.sub(' ', text)
    # quitar menciones
    text = MENTION_RE.sub(' ', text)
    # convertir hashtags a la palabra (ej: #amor -> amor) o borrarlos
    if keep_hashtags:
        text = HASHTAG_RE.sub(r'\1', text)
    else:
        text = HASHTAG_RE.sub(' ', text)
    # primero demojize (cubre la mayoría)
    text = demojize_and_map(text)
    # luego mapea símbolos extra (♬ ❤ ...)
    text = replace_symbol_tokens(text)
    # emoticonos de texto
    text = replace_emoticons(text)
    # elimina cualquier pictograma restante
    text = UNICODE_SYMBOLS_RE.sub(' ', text)
    # quitar caracteres basura y normalizar espacios
    text = remove_junk_chars(text)
    text = EXCESS_WS_RE.sub(' ', text).strip()
    if lower:
        text = text.lower()
    
    # Se mantiene por defecto no eliminar números ni toda la puntuación para no perder señales.
    text = clean(text, fix_unicode=False, to_ascii=False, lower=False, no_urls=True, no_emails=True,
                 no_phone_numbers=True, no_numbers=False, no_punct=False)
    # último barrido de espacios
    text = EXCESS_WS_RE.sub(' ', text).strip()
    return text

def clean_dataframe(df, text_col='body', inplace=False, **kwargs):
    if not inplace:
        df = df.copy()
    df['clean_' + text_col] = df[text_col].fillna('').apply(lambda x: clean_post(x, **kwargs))
    return df

# Uso:
df = pd.read_csv('material/posts.csv')  # tu CSV
df = clean_dataframe(df, text_col='body', inplace=False, lower=True, keep_hashtags=True)
df.to_csv('material/posts_clean.csv', index=False)


In [2]:
# Asegurarse de que las columnas estén bien nombradas
df.columns = df.columns.str.strip()  # elimina espacios si los hubiera

# Limpiar NaN o textos vacíos
df['clean_body'] = df['clean_body'].fillna('').astype(str)

# Agrupar por autor
df_grouped = (
    df.groupby('username', as_index=False)
      .agg({'clean_body': list})   # convierte los posts de cada autor en lista
)

# Ahora cada fila tiene una lista de strings
print(df_grouped.head())
df_grouped.to_csv('material/posts_clean_by_author.csv', index=False)


       username                                         clean_body
0   -Areopagan-  [your first and second question is the same qu...
1     -BigSexy-  [i've been asked to cum everywhere with my ex ...
2    -BlitzN9ne  [i'm currently in the middle of making a payda...
3  -CrestiaBell  [first and foremost i extend my condolences to...
4        -dyad-  [i failed both . i'm great at reading people i...


Tenemos los posts en una lista. Solo necesitamos dividir los posts de cada usuario en oraciones, para posteriormente quedarnos con las más representativas.

📝 **Nota**: Al principio, utilizamos el módulo `spacy`, pero al tener más de 3 millones de posts la ejecución era demasiado larga. La siguiente solución es más rápida, pero menos precisa.

In [4]:
def quick_split(posts):
    sentences = []
    for post in posts:
        parts = re.split(r'(?<=[.!?])\s+', post.strip())
        sentences.extend([p for p in parts if p])
    return sentences

df_grouped['clean_body'] = df_grouped['clean_body'].apply(quick_split)

# Ahora cada fila tiene una lista de oraciones para cada autor
print(df_grouped.head())
df_grouped.to_csv('material/posts_clean_by_author.csv', index=False)

       username                                         clean_body
0   -Areopagan-  [your first and second question is the same qu...
1     -BigSexy-  [i've been asked to cum everywhere with my ex ...
2    -BlitzN9ne  [i'm currently in the middle of making a payda...
3  -CrestiaBell  [first and foremost i extend my condolences to...
4        -dyad-  [i failed both ., i'm great at reading people ...
